In [1]:
import pandas as pd

In [25]:
filepath = 'data/eurusd_hour.csv'

data = pd.read_csv(filepath_or_buffer=filepath, encoding='utf-8', parse_dates=["Date"])

In [26]:
print(data.head())
print(data.shape[0])
# pd.to_datetime(data)
print(data.info())

        Date   Time      BO      BH      BL      BC     BCh      AO      AH  \
0 2005-05-02  00:00  1.2852  1.2852  1.2840  1.2844 -0.0008  1.2854  1.2854   
1 2005-05-02  01:00  1.2844  1.2848  1.2839  1.2842 -0.0002  1.2846  1.2850   
2 2005-05-02  02:00  1.2843  1.2854  1.2841  1.2851  0.0008  1.2845  1.2856   
3 2005-05-02  03:00  1.2851  1.2859  1.2850  1.2851  0.0000  1.2853  1.2861   
4 2005-05-02  04:00  1.2852  1.2859  1.2849  1.2855  0.0003  1.2854  1.2861   

       AL      AC     ACh  
0  1.2842  1.2846 -0.0008  
1  1.2841  1.2844 -0.0002  
2  1.2843  1.2853  0.0008  
3  1.2852  1.2853  0.0000  
4  1.2851  1.2857  0.0003  
93084
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93084 entries, 0 to 93083
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    93084 non-null  datetime64[ns]
 1   Time    93084 non-null  object        
 2   BO      93084 non-null  float64       
 3   BH      93084 non

In [31]:
train = data.query('("2005-01-01" <= Date < "2015-01-01")').copy()
test = data.query('("2015-01-01" <= Date < "2021-01-01")').copy()
print(train.shape[0])
print(test.shape[0])
assert data.shape[0] == (train.shape[0] + test.shape[0])
# 2005-05-02
# 2020-04-29

60223
32861


In [1]:
from classes.EnvFactory import EnvFactory

In [2]:
train, test = EnvFactory().DL.dataset_train_test
print(train.shape[0])
print(test.shape[0])

60223
32861


In [8]:
from collections import deque
hist_reward = deque()

hist_reward.append(0)
hist_reward.append(1)
hist_reward.append(2)

In [10]:
import pandas as pd
a = pd.DataFrame({
    'aze' : hist_reward
})
a.head()

,aze
0,0
1,1
2,2


In [16]:
from classes.ActionScheme import *


a= RandomAction().compute_action()
a

1

In [1]:
from classes.ForexEnv import ForexEnv
import gymnasium as gym 

In [3]:
gym.register(
    id='forexEnv-V0',
    entry_point = 'ForexEnv:ForexEnv'

)

In [1]:
import comet_ml
import gymnasium as gym 
from stable_baselines3 import PPO, A2C, DQN
from classes.ForexEnv import ForexEnv
from classes.DataLoader import DataLoader
from comet_ml.integration.gymnasium import CometLogger

In [2]:
data, _ = DataLoader(filepath='data/eurusd_minute.csv').dataset_train_test
# data, _ = DataLoader(filepath='data/eurusd_minute.csv').dataset_train_test

In [3]:
env = ForexEnv(data)

experiment = comet_ml.Experiment(
  api_key="WVdjqWHN6SXFKqIlH5H8SnumS",
  project_name="general",
  workspace="biwal"
)

env = CometLogger(env, experiment)

model = PPO("MlpPolicy", env, verbose=0 ,  seed=1,
            batch_size=256,
            gamma=0.9999,
            learning_rate=2.77e-05,
            ent_coef=0.00429,
            clip_range=0.1,
            n_epochs=10,
            gae_lambda=0.9,
            max_grad_norm=5,
            vf_coef=0.19)
model.learn(total_timesteps=50_000)

env.close()
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in 'c:\\env_py\\PERSO_TradingBot2' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/biwal/general/997ae26716d74a77b211d36559001c1f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/biwal/general/997ae26716d74a77b211d36559001c1f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     episode_length       : 251
COMET INFO:     episode_reward [203] : (-28.317

In [5]:
#  Test Model

env = ForexEnv(_, episodic=False)
obs = env.reset()[0]
for i in range(2000):
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, terminated, truncated, info = env.step(action)


    if terminated:
        env.render()
        obs = env.reset()[0]
env.render()

 - Suivi de la récompense  :
        Nombre de récompenses enregistrées : 2000 
        Récompense cumulé : -1.901e+01
        Récompense moyenne : -0.010 
        
